In [ ]:
import pandas as pd
import numpy as np


In [ ]:

# Читаем файл
df = pd.read_csv(r'C:\Users\Stukova_a\Documents\competitors_price_faces_KSA.txt', 
                 delimiter=';', 
                 encoding='cp1251')

# Находим колонки, которые содержат '|'
for column in df.columns:
    if df[column].dtype == 'object':  # Проверяем только текстовые колонки
        if df[column].str.contains('|', regex=False).any():
            # Разбиваем колонку на несколько новых
            split_cols = df[column].str.split('|', expand=True)
            
            # Называем новые колонки
            for i in range(len(split_cols.columns)):
                new_col_name = f"{column}_{i+1}"
                df[new_col_name] = split_cols[i]
            
            # Удаляем исходную колонку
            df = df.drop(columns=[column])


In [ ]:
# Создаем словарь для переименования колонок
df.columns = ['Пусто', 'Дата', 'Категория', 'Бренд', 'Наличие', 'Цена1', 'Цена2','Пусто' ]

# Удаляем колонки _7 и _8
#df = df.drop(['|marketCollected |category1|marketDiscountPrice|marketBrandGoldenApple |marketPriceValue|marketAvailability|_7',
 #            '|marketCollected |category1|marketDiscountPrice|marketBrandGoldenApple |marketPriceValue|marketAvailability|_8'], axis=1)

# Выводим первые строки датафрейма для проверки
print(df.head())

In [ ]:
df

In [ ]:
df = df.drop(0)

# Удаляем первую и последнюю колонки
df = df.drop(['Пусто', 'Наличие'], axis=1)

print(df.head())

In [ ]:
df['Цена1'] = pd.to_numeric(df['Цена1'], errors='coerce')
df['Цена2'] = pd.to_numeric(df['Цена2'], errors='coerce')

# Вычисляем скидку в процентах
df['Скидка %'] = ((df['Цена2'] - df['Цена1']) / df['Цена2'] * 100).round(2)

print(df.head())

In [ ]:
# Добавляем столбцы с неделей и месяцем года
df['Неделя года'] = pd.to_datetime(df['Дата']).dt.isocalendar().week
df['Месяц'] = pd.to_datetime(df['Дата']).dt.month

pivot_df = pd.pivot_table(df,
                        values=['Скидка %'],
                        index=['Дата', 'Бренд', 'Неделя года', 'Месяц'],
                        aggfunc={
                            'Скидка %': ['mean', 'count']
                        }).round(2)

pivot_df.columns = ['Средняя скидка', 'Количество SKU']
pivot_df = pivot_df.reset_index()

# Сохраняем в Excel
# pivot_df.to_excel('sephora_daily_brand_analysis.xlsx', index=False)

print(pivot_df.head())

In [ ]:
# Оставляем только нужные колонки
df = df[['Дата', 'Бренд', 'Скидка %', 'Неделя года', 'Месяц']]

print(df.head())


In [ ]:
df['Дата'] = pd.to_datetime(df['Дата']).dt.date

In [ ]:
df

In [ ]:

# Преобразуем дату в формат даты и сортируем
df['Дата'] = pd.to_datetime(df['Дата']).dt.date
df = df.sort_values(['Месяц', 'Дата'])

# Проверяем уникальные месяцы
print("Уникальные месяцы в данных:")
print(df['Месяц'].unique())

# Проверяем диапазон дат
print("\nДиапазон дат:")
print("Начало:", df['Дата'].min())
print("Конец:", df['Дата'].max())




In [ ]:
df_discounted = df[df['Скидка %'] > 0]

discount_analysis = pd.pivot_table(df_discounted,
                                 index=['Бренд', 'Неделя года', 'Месяц'],
                                 values='Скидка %',
                                 aggfunc=['mean', 'count'])

discount_analysis.columns = ['Средняя скидка', 'Количество SKU']
discount_analysis = discount_analysis.round(2).reset_index()


# Создаем словарь с названиями месяцев
month_names = {
   1: 'Январь',
   2: 'Февраль',
   3: 'Март',
   4: 'Апрель',
   5: 'Май',
   6: 'Июнь',
   7: 'Июль',
   8: 'Август',
   9: 'Сентябрь',
   10: 'Октябрь',
   11: 'Ноябрь',
   12: 'Декабрь'
}

# Заменяем цифры на названия месяцев
discount_analysis['Месяц'] = discount_analysis['Месяц'].map(month_names)

print(discount_analysis.head())




In [ ]:

discount_analysis.to_excel(r'C:\Users\Stukova_a\Documents\discount_analysis_faces_KSA.xlsx')

print(discount_analysis)


In [ ]:
# Создаем второй DataFrame с нужным форматом
monthly_analysis = pd.pivot_table(df_discounted,
                               index=['Месяц', 'Неделя года'],
                               values='Скидка %',
                               aggfunc={'Скидка %': ['mean', 'count']}).round(2)

monthly_analysis.columns = ['Средняя скидка', 'Количество SKU на промо']
monthly_analysis = monthly_analysis.reset_index()

# Заменяем цифры месяцев на названия
monthly_analysis['Месяц'] = monthly_analysis['Месяц'].map(month_names)

# Сортируем по месяцу и неделе
monthly_analysis = monthly_analysis.sort_values(['Месяц', 'Неделя года'])

# Записываем оба DataFrame в разные листы одного файла
with pd.ExcelWriter(r'C:\Users\Stukova_a\Documents\discount_analysis_faces_KSA.xlsx') as writer:
   discount_analysis.to_excel(writer, sheet_name='Brand_Analysis', index=False)
   monthly_analysis.to_excel(writer, sheet_name='Monthly_Analysis', index=False)